# IBM Data Science Capstone Project

### A project by Bangkit Dana Setiawan
**_Opening a New Korean Restaurant South Jakarta, Indonesia**

- Build a dataframe of neighborhoods in South Jakarta, Malaysia by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Cafe
***


### 1. Import libraries

In [12]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!pip install geocoder
import geocoder # to get coordinates

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 17.0MB/s ta 0:00:01
Folium installed
Libraries imported.


In [11]:
! pip install BeautifulSoup4

from bs4 import BeautifulSoup # library to parse HTML and XML documents

### 2. Scrap Data from Wikipedia Page into a DataFrame

In [14]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:South_Jakarta").text

In [15]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [16]:
# create a list to store neighborhood data
neighborhoodList = []

In [17]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [18]:
# create a new DataFrame from the list
sj_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sj_df.head()

,Neighborhood
0,South Jakarta
1,AIS Indonesia
2,ASEAN MRT station
3,Atma Jaya Catholic University of Indonesia
4,Balai Sarbini


In [19]:
# print the number of rows of the dataframe
sj_df.shape

(63, 1)

### 3. Get the Geographical Coordinates

In [20]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, South Jakarta, Indonesia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [21]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in sj_df["Neighborhood"].tolist() ]

In [22]:
coords

[[-6.253739999999937, 106.79731000000004],
 [-6.278239999999926, 106.82858000000004],
 [-6.166239999999959, 106.87657000000007],
 [-6.194169999999929, 106.84914000000003],
 [-6.253739999999937, 106.79731000000004],
 [-6.236189999999965, 106.75495000000006],
 [-6.157030600195227, 106.77319929521403],
 [-6.253739999999937, 106.79731000000004],
 [-6.232009999999946, 106.82067000000006],
 [-6.2905099999999265, 106.79491000000007],
 [-6.253739999999937, 106.79731000000004],
 [-6.243109999999945, 106.84359000000006],
 [-6.255259999999964, 106.85533000000004],
 [-6.2925299999999424, 106.79340000000008],
 [-6.339802847864294, 106.85198502543803],
 [-6.245719999999949, 106.78393000000005],
 [-6.139409999999941, 106.80925000000008],
 [-6.343079999999929, 106.81745000000006],
 [-6.286059999999964, 106.78253000000007],
 [-6.211364796633448, 106.83052005564355],
 [-6.215470475445661, 106.81870459830499],
 [-6.279909999999973, 106.81650000000008],
 [-6.227639999999951, 106.82692000000009],
 [-6.2514

In [23]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [24]:
# merge the coordinates into the original dataframe
sj_df['Latitude'] = df_coords['Latitude']
sj_df['Longitude'] = df_coords['Longitude']

In [25]:
# check the neighborhoods and the coordinates
print(sj_df.shape)
sj_df

(63, 3)


,Neighborhood,Latitude,Longitude
0,South Jakarta,-6.253740,106.797310
1,AIS Indonesia,-6.278240,106.828580
2,ASEAN MRT station,-6.166240,106.876570
3,Atma Jaya Catholic University of Indonesia,-6.194170,106.849140
4,Balai Sarbini,-6.253740,106.797310
5,Bintaro Jaya,-6.236190,106.754950
6,"Blok M, Jakarta",-6.157031,106.773199
7,BTPN Towers,-6.253740,106.797310
8,Capital Place Jakarta,-6.232010,106.820670
9,Cilandak,-6.290510,106.794910


In [26]:
# save the DataFrame as CSV file
sj_df.to_csv("sj_df.csv", index=False)

### 4. Create a Map of South Jakarta with Neighborhoods Superimposed on Top

In [30]:
# get the coordinates of South Jakarta
address = 'South Jakarta, Indonesia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of South Jakarta are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of South Jakarta are -6.2639932, 106.7924085.


In [31]:
# create map of South Jakarta using latitude and longitude values
map_sj = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(sj_df['Latitude'], sj_df['Longitude'], sj_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sj)  
    
map_sj

In [32]:
# save the map as HTML file
map_sj.save('map_kl.html')

### 5. Use the Foursquare API to Explore the Neighborhoods

In [33]:
# define Foursquare Credentials and Version
CLIENT_ID = '5UL0MKKRCYWAN0ZUSFR1VQXCC4IU1ZWFOEPGQOIYALN3MGGW' # your Foursquare ID
CLIENT_SECRET = 'C3DWJ1SHWGF3OJPLTCP2AZR5NNH0FJR3OFJUP3S0RUFRZEI2' # your Foursquare Secret
VERSION = '20200604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5UL0MKKRCYWAN0ZUSFR1VQXCC4IU1ZWFOEPGQOIYALN3MGGW
CLIENT_SECRET:C3DWJ1SHWGF3OJPLTCP2AZR5NNH0FJR3OFJUP3S0RUFRZEI2


Then, let's get the top 100 venues that are within a radius of 2000 meters

In [34]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sj_df['Latitude'], sj_df['Longitude'], sj_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [35]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5928, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,South Jakarta,-6.25374,106.79731,Nasi Uduk Pak Joko Putro,-6.254641,106.796036,Indonesian Restaurant
1,South Jakarta,-6.25374,106.79731,Sopo Ngiro Japanese Food,-6.251255,106.797441,Japanese Restaurant
2,South Jakarta,-6.25374,106.79731,Nasi Campur Kenanga,-6.253358,106.800189,BBQ Joint
3,South Jakarta,-6.25374,106.79731,Chung Gi Wa 청기와,-6.252591,106.800207,Korean Restaurant
4,South Jakarta,-6.25374,106.79731,All Fresh,-6.249712,106.797916,Fruit & Vegetable Store


Let's check how many venues were returned for each neighorhood

In [36]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
AIS Indonesia,100,100,100,100,100,100
ASEAN MRT station,100,100,100,100,100,100
Atma Jaya Catholic University of Indonesia,100,100,100,100,100,100
BTPN Towers,100,100,100,100,100,100
Balai Sarbini,100,100,100,100,100,100
Bintaro Jaya,59,59,59,59,59,59
"Blok M, Jakarta",100,100,100,100,100,100
Capital Place Jakarta,100,100,100,100,100,100
Cilandak,100,100,100,100,100,100



Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


In [38]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indonesian Restaurant', 'Japanese Restaurant', 'BBQ Joint',
       'Korean Restaurant', 'Fruit & Vegetable Store',
       'General Entertainment', 'Seafood Restaurant', "Women's Store",
       'Board Shop', 'Hotel', 'Burger Joint', 'Coffee Shop',
       'Snack Place', 'Spa', 'Gourmet Shop', 'Noodle House',
       'Theme Restaurant', 'Comfort Food Restaurant', 'Restaurant',
       'Italian Restaurant', 'Sushi Restaurant', 'Fried Chicken Joint',
       'Food Truck', 'Supermarket', 'Beer Garden', 'Sporting Goods Shop',
       'Food & Drink Shop', 'Grocery Store', 'Lounge',
       'Furniture / Home Store', 'Steakhouse', 'Pizza Place',
       'Breakfast Spot', 'Automotive Shop', 'Art Gallery',
       'German Restaurant', 'Shopping Mall', 'Manadonese Restaurant',
       'Soup Place', 'Juice Bar', 'Asian Restaurant', 'Bakery',
       'Playground', 'Electronics Store', 'Smoke Shop', 'Plaza', 'Bar',
       'Salon / Barbershop', 'Clothing Store', 'Javanese Restaurant'],
      dtype=objec

In [39]:
# check if the results contain "Korean Restaurant"
"Korean Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyzing Each Neighborhoods

In [41]:
# one hot encoding
sj_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sj_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sj_onehot.columns[-1]] + list(sj_onehot.columns[:-1])
sj_onehot = sj_onehot[fixed_columns]

print(sj_onehot.shape)
sj_onehot.head()

(5928, 238)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Event Space,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Library,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,South Jakarta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Sou

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
sj_grouped = sj_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sj_grouped.shape)
sj_grouped

(63, 238)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Event Space,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Library,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,AIS Indonesia,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.00,0.030000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.00,0.010000,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.030000,0.01,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.140000,0.00,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.020000,0.010000,0.010000,0.000000,0.0

In [44]:
len(sj_grouped[sj_grouped["Korean Restaurant"] > 0])

34

Create a new DataFrame for Korean Restaurant data only

In [45]:
sj_korean = sj_grouped[["Neighborhoods","Korean Restaurant"]]

In [46]:
sj_korean.head()

,Neighborhoods,Korean Restaurant
0,AIS Indonesia,0.00
1,ASEAN MRT station,0.00
2,Atma Jaya Catholic University of Indonesia,0.00
3,BTPN Towers,0.01
4,Balai Sarbini,0.01


### 7. Cluster Neighborhoods

In [47]:
# set number of clusters
kclusters = 3

sj_clustering = sj_korean.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sj_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 2, 0, 2, 1, 0, 2], dtype=int32)

In [48]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sj_merged = sj_korean.copy()

# add clustering labels
sj_merged["Cluster Labels"] = kmeans.labels_

In [49]:
sj_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sj_merged.head()

,Neighborhood,Korean Restaurant,Cluster Labels
0,AIS Indonesia,0.00,0
1,ASEAN MRT station,0.00,0
2,Atma Jaya Catholic University of Indonesia,0.00,0
3,BTPN Towers,0.01,2
4,Balai Sarbini,0.01,2


In [50]:
# merge sj_grouped with sj_data to add latitude/longitude for each neighborhood
sj_merged = sj_merged.join(sj_df.set_index("Neighborhood"), on="Neighborhood")

print(sj_merged.shape)
sj_merged.head() # check the last columns!

(63, 5)


,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
0,AIS Indonesia,0.00,0,-6.27824,106.82858
1,ASEAN MRT station,0.00,0,-6.16624,106.87657
2,Atma Jaya Catholic University of Indonesia,0.00,0,-6.19417,106.84914
3,BTPN Towers,0.01,2,-6.25374,106.79731
4,Balai Sarbini,0.01,2,-6.25374,106.79731


In [51]:
# sort the results by Cluster Labels
print(sj_merged.shape)
sj_merged.sort_values(["Cluster Labels"], inplace=True)
sj_merged

(63, 5)


,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
0,AIS Indonesia,0.000000,0,-6.278240,106.828580
28,Lebak Bulus Grab MRT station,0.000000,0,-6.289210,106.776030
37,"Pancoran, South Jakarta",0.000000,0,-6.255480,106.843910
38,Pasar Minggu Baru railway station,0.000000,0,-6.171480,106.826490
39,Pasar Minggu railway station,0.000000,0,-6.292650,106.827910
41,Pejaten Village,0.000000,0,-6.280500,106.829050
43,"Pesanggrahan, South Jakarta",0.000000,0,-6.236400,106.754470
21,Jeruk Purut Cemetery,0.000000,0,-6.279910,106.816500
44,Pondok Indah,0.000000,0,-6.268570,106.780560
45,Pondok Indah Mall,0.000000,0,-6.266260,106.783350


Finally, let's visualize the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sj_merged['Latitude'], sj_merged['Longitude'], sj_merged['Neighborhood'], sj_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

Cluster 0

In [55]:
sj_merged.loc[sj_merged['Cluster Labels'] == 0]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
0,AIS Indonesia,0.0,0,-6.278240,106.828580
28,Lebak Bulus Grab MRT station,0.0,0,-6.289210,106.776030
37,"Pancoran, South Jakarta",0.0,0,-6.255480,106.843910
38,Pasar Minggu Baru railway station,0.0,0,-6.171480,106.826490
39,Pasar Minggu railway station,0.0,0,-6.292650,106.827910
41,Pejaten Village,0.0,0,-6.280500,106.829050
43,"Pesanggrahan, South Jakarta",0.0,0,-6.236400,106.754470
21,Jeruk Purut Cemetery,0.0,0,-6.279910,106.816500
44,Pondok Indah,0.0,0,-6.268570,106.780560
45,Pondok Indah Mall,0.0,0,-6.266260,106.783350


Cluster 1

In [56]:
sj_merged.loc[sj_merged['Cluster Labels'] == 1]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
55,Sudirman Central Business District,0.06,1,-6.22577,106.81183
36,Pacific Place Jakarta,0.06,1,-6.22450,106.80974
62,The Plaza Semanggi,0.04,1,-6.21959,106.81440
7,Capital Place Jakarta,0.05,1,-6.23201,106.82067


Cluster 2

In [57]:
sj_merged.loc[sj_merged['Cluster Labels'] == 2]

,Neighborhood,Korean Restaurant,Cluster Labels,Latitude,Longitude
30,List of regencies and cities in Jakarta,0.010000,2,-6.253740,106.797310
60,The Pakubuwono,0.020000,2,-6.234881,106.791954
59,"Tebet, South Jakarta",0.010000,2,-6.229390,106.849710
58,Tebet railway station,0.010000,2,-6.229390,106.849710
3,BTPN Towers,0.010000,2,-6.253740,106.797310
4,Balai Sarbini,0.010000,2,-6.253740,106.797310
6,"Blok M, Jakarta",0.010000,2,-6.157031,106.773199
53,South Jakarta,0.010000,2,-6.253740,106.797310
52,"Singapore International School, Indonesia",0.011236,2,-6.197270,106.758760
9,Ciputra World Jakarta,0.010000,2,-6.253740,106.797310


### Observations:
Most of the korean restaurant are concentrated in the south jakarta, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no korean restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new korean restaurant as there is very little to no competition from existing restaurant. Meanwhile, korean restaurant in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of korean restaurant. From another perspective, this also shows that the oversupply of korean restaurant mostly happened in the central area of the city, with the suburb area still have very few restaurant. Therefore, this project recommends property developers to capitalize on these findings to open new korean restaurant in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new korean restaurant in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of korean restaurant and suffering from intense competition.